In [1]:
import sys
import os

# Aggiungi il percorso del progetto al PYTHONPATH
sys.path.append(os.path.abspath(r"C:\Users\cical\Documents\GitHub\Repositories\pig_tissue_segmentation"))

import src.data.make_dataset as make_dataset

slice_list = make_dataset.load_image_slices(r"C:\Users\cical\Documents\GitHub\Repositories\pig_tissue_segmentation\data\raw")

for slice in slice_list:
    slice.analyse_image()
